# Microsoft Binary Suspicious Communication Endpoint
Detects an executable in the Windows folder accessing suspicious domains

## Rule Content
```
- title: Microsoft Binary Suspicious Communication Endpoint
  id: e0f8ab85-0ac9-423b-a73a-81b3c7b1aa97
  status: experimental
  description: Detects an executable in the Windows folder accessing suspicious domains
  references:
  - https://twitter.com/M_haggis/status/900741347035889665
  - https://twitter.com/M_haggis/status/1032799638213066752
  author: Florian Roth
  date: 2018/08/30
  tags:
  - attack.lateral_movement
  - attack.t1105
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection:
      EventID: 3
      Initiated: 'true'
      DestinationHostname:
      - '*dl.dropboxusercontent.com'
      - '*.pastebin.com'
      - '*.githubusercontent.com'
      Image: C:\Windows\\*
    condition: selection
  falsepositives:
  - Unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"3" AND network_initiated:"true" AND dst_host_name.keyword:(*dl.dropboxusercontent.com OR *.pastebin.com OR *.githubusercontent.com) AND process_path.keyword:C\:\\Windows\\*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()